In [9]:
import pandas as pd
import os
import numpy as np
from glob import glob

os.chdir('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/code')

In [ ]:
flair_flavor_list = ['AX T2 FLAIR', 'Axial T2 FLAIR', 'SAG T2 FLAIR', 'AX FLAIR',
 'COR FLAIR', 'SAG 3D FLAIR', '3D Sag T2 FLAIR Cube']
flair_flavor_list += [flavor.replace(' ', '_') for flavor in flair_flavor_list]
flair_flavor_list += [flavor.replace(' ', '-') for flavor in flair_flavor_list]
flair_flavor_list = [flavor.lower() for flavor in flair_flavor_list]
flair_flavor_list

In [ ]:
def is_flair(x):
    global flair_flavor_list
    res = x.lower()
    for flavor in flair_flavor_list:
        if flavor in res:
            return True
    
    return False

In [ ]:
t1_processed_file_df = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/experiments/20211102/volume_label_with_file_path_with_age_20211102.tsv', sep='\t', index_col=0)
t1_processed_file_df = t1_processed_file_df.groupby('Session').nth(0)
t1_processed_file_df.reset_index(inplace=True)
t1_processed_file_df

In [ ]:
# Scan file of all modalities
all_file = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/code/all_file_path_with_label.tsv', sep='\t')
all_file.drop(['scan_date_time', 'label'], axis=1, inplace=True)
# Scan file of T1 modalities
t1_file = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/code/t1_file_path_with_label.tsv', sep='\t')
t1_file.rename({'Path': 't1_path'}, axis=1, inplace=True)
# We only need one t1 file for each session to make flair registration
t1_file = t1_file.groupby('Session').nth(0)
t1_file.reset_index(inplace=True)
# get exact t1 pre-processed file path
t1_processed_file_df = pd.read_csv('/gpfs/home/lc3424/capstone/2021_dementia/lc3424_workspace/experiments/20211102/volume_label_with_file_path_with_age_20211102.tsv', sep='\t', index_col=0)
t1_processed_file_df = t1_processed_file_df.groupby('Session').nth(0)
t1_processed_file_df.reset_index(inplace=True)
t1_file = t1_file.merge(t1_processed_file_df, on=['Session', 'Subject'], how='right')
t1_file.drop(['label', 'dest_path', 't1_path', 'Age'], axis=1, inplace=True)
t1_file.rename({'Path': "t1_path"}, inplace=True, axis=1)
t1_file

In [ ]:
flair_file = all_file.copy()
flair_file = flair_file[flair_file.Path.apply(is_flair)]
flair_file.reset_index(drop=True, inplace=True)
flair_file.rename({'Path': 'flair_path'}, axis=1, inplace=True)
flair_file

In [ ]:
t1_flair_file = flair_file.merge(t1_file, on=['Session', 'Subject'], how='left')
# print(t1_flair_file.shape)
t1_flair_file.dropna(axis=0, inplace=True)

t1_flair_file

In [ ]:
t1_flair_file.to_csv('t1_flair_file_match.tsv', sep='\t', index=False)

In [10]:
t1_flair_file = pd.read_csv('t1_flair_file_match.tsv', sep='\t')

In [12]:
temp = t1_flair_file.loc[1, ['flair_path', 't1_path']]
temp

flair_path    /gpfs/data/razavianlab/data/mri/nyu/barlow_nii...
t1_path       /gpfs/data/razavianlab/data/mri/nyu/barlow_bid...
Name: 1, dtype: object

In [13]:
flair, t1 = temp.flair_path, temp.t1_path